In [5]:
import joblib

In [6]:
%matplotlib inline
 
import matplotlib.pyplot as plt
import numpy as np
import os
import pprint


In [7]:
from skimage.io import imread
from skimage.transform import resize

In [4]:
from zipfile import ZipFile
file_name = "Project_Mask_Detection.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [5]:
def resize_all(src, pklname, include, width=300, height=None):
    """
    load images from path, resize them and write them as arrays to a dictionary, 
    together with labels and metadata. The dictionary is written to a pickle file 
    named '{pklname}_{width}x{height}px.pkl'.
     
    Parameter
    ---------
    src: str
        path to data
    pklname: str
        path to output file
    width: int
        target width of the image in pixels
    include: set[str]
        set containing str
    """
     
    height = height if height is not None else width
     
    data = dict()
    data['description'] = 'resized ({0}x{1})animal images in rgb'.format(int(width), int(height))
    data['label'] = []
    data['filename'] = []
    data['data'] = []   
     
    pklname = f"{pklname}_{width}x{height}px.pkl"
 
    # read all images in PATH, resize and write to DESTINATION_PATH
    for subdir in os.listdir(src):
        if subdir in include:
            print(subdir)
            current_path = os.path.join(src, subdir)
 
            for file in os.listdir(current_path):
                if file[-3:] in {'jpg', 'png'}:
                    im = imread(os.path.join(current_path, file))
                    im = resize(im, (width, height)) #[:,:,::-1]
                    data['label'].append(int(subdir[:1]))
                    data['filename'].append(file)
                    data['data'].append(im)
 
        joblib.dump(data, pklname)

In [6]:
resize_all(src='Project_Mask_Detection/maskdata/maskdata/train', pklname='train_set', include={'1','0'}, width=300)

0
1


In [7]:
resize_all(src='Project_Mask_Detection/maskdata/maskdata/test', pklname='test_set', include={'1','0'}, width=300)

0
1


In [8]:
from collections import Counter
data = joblib.load(f'train_set_300x300px.pkl')
X_train = np.array(data['data'])
Y_train = np.array(data['label'])
Y_train = Y_train.reshape(-1,1)
print('number of samples: ', len(data['data']))
print('keys: ', list(data.keys()))
print('description: ', data['description'])
print('image shape: ', data['data'][0].shape)
print('labels:', np.unique(Y_train))
 
Counter(data['label'])
print(Y_train.shape)

number of samples:  616
keys:  ['description', 'label', 'filename', 'data']
description:  resized (300x300)animal images in rgb
image shape:  (300, 300, 3)
labels: [0 1]
(616, 1)


In [9]:
Y_train[274]

array([0])

In [10]:
data_test = joblib.load(f'test_set_300x300px.pkl')
 
X_test = np.array(data_test['data'])
Y_test = data_test['label']
Y_test = np.array(Y_test)
Y_test = Y_test.reshape(-1,1)
print('number of samples: ', len(data_test['data']))
print('keys: ', list(data_test.keys()))
print('description: ', data_test['description'])
print('image shape: ', data_test['data'][0].shape)
print('labels:', np.unique(Y_test))

Counter(data_test['label'])
Y_test.shape

number of samples:  198
keys:  ['description', 'label', 'filename', 'data']
description:  resized (300x300)animal images in rgb
image shape:  (300, 300, 3)
labels: [0 1]


(198, 1)

In [14]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [15]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [16]:
early_stop = EarlyStopping(monitor= 'val_loss',mode ='min',verbose=1, patience = 3)

In [17]:
model.fit(x=X_train, y=Y_train, validation_data=(X_test,Y_test), batch_size=25, epochs = 60,callbacks=[early_stop])

Epoch 1/60
25/25 [==============================] - 139s 5s/step - loss: 6.3843 - acc: 0.4740 - val_loss: 0.6936 - val_acc: 0.3333
Epoch 2/60
25/25 [==============================] - 123s 5s/step - loss: 0.6911 - acc: 0.5410 - val_loss: 0.7321 - val_acc: 0.3384
Epoch 3/60
25/25 [==============================] - 122s 5s/step - loss: 0.8554 - acc: 0.6244 - val_loss: 0.5277 - val_acc: 0.8283
Epoch 4/60
25/25 [==============================] - 124s 5s/step - loss: 0.4117 - acc: 0.8448 - val_loss: 0.7644 - val_acc: 0.5455
Epoch 5/60
25/25 [==============================] - 127s 5s/step - loss: 0.2630 - acc: 0.9124 - val_loss: 0.2476 - val_acc: 0.8838
Epoch 6/60
25/25 [==============================] - 123s 5s/step - loss: 0.1467 - acc: 0.9514 - val_loss: 0.1852 - val_acc: 0.9444
Epoch 7/60
25/25 [==============================] - 113s 5s/step - loss: 0.2356 - acc: 0.9245 - val_loss: 0.1168 - val_acc: 0.9596
Epoch 8/60
25/25 [==============================] - 112s 4s/step - loss: 0.1162 - a

In [41]:
'''import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()'''

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [18]:
import numpy as np
import cv2
from keras.models import load_model

In [19]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)
  #  model = load_model('model-021.model')
    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(300,300))
        reshaped=np.reshape(resized,(1,300,300,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()